# NN Train on Simulated Data

This notebook trains a Neural Network using the pre-processed simulated data. A mean accuracy of 98% is achieved by running this notebook several times.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
from torch.autograd import Variable
from torch.optim.lr_scheduler import ExponentialLR

from sklearn.model_selection import train_test_split
from sklearn import metrics
import seaborn as sns


n_outputs, p_outputs = ("Volt_n1_V_out","Volt_n2_V_out","Volt_n3_V_out"),("Volt_p1_V_out","Volt_p2_V_out","Volt_p3_V_out")

headers = ["n1","n2","n3",
           "p1","p2","p3",
           "AP1", "AP2", "AP3", "AP4", "AP5", "AP6", "AP7", "AP8", "AP9"] # AP == active point i

In [51]:
df = pd.read_csv("Data/normalized_data.csv", usecols = headers)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   n1      511 non-null    float64
 1   n2      511 non-null    float64
 2   n3      511 non-null    float64
 3   p1      511 non-null    float64
 4   p2      511 non-null    float64
 5   p3      511 non-null    float64
 6   AP1     511 non-null    int64  
 7   AP2     511 non-null    int64  
 8   AP3     511 non-null    int64  
 9   AP4     511 non-null    int64  
 10  AP5     511 non-null    int64  
 11  AP6     511 non-null    int64  
 12  AP7     511 non-null    int64  
 13  AP8     511 non-null    int64  
 14  AP9     511 non-null    int64  
dtypes: float64(6), int64(9)
memory usage: 60.0 KB


In [53]:
df.head()

,n1,n2,n3,p1,p2,p3,AP1,AP2,AP3,AP4,AP5,AP6,AP7,AP8,AP9
0,-0.462344,-0.262758,-0.176811,0.758348,0.697728,0.665282,1,1,1,1,1,1,1,1,1
1,-0.395858,-0.261516,-0.190474,0.738963,0.681015,0.653584,0,1,1,1,1,1,1,1,1
2,-0.480484,-0.217668,-0.194590,0.679906,0.688107,0.682851,1,0,1,1,1,1,1,1,1
3,-0.475069,-0.272240,-0.119205,0.064385,0.783911,0.723635,1,1,0,1,1,1,1,1,1
4,-0.281580,-0.280480,-0.208023,0.741358,0.665297,0.635337,1,1,1,0,1,1,1,1,1


In [54]:
#np.vstack(df_scaled["AP"].values) #stacks list of the arrays into single array

x = df.drop(columns = headers[6:])
y = df.drop(columns = headers[:6])
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [55]:
df.abs().max()

n1     1.0
n2     1.0
n3     1.0
p1     1.0
p2     1.0
p3     1.0
AP1    1.0
AP2    1.0
AP3    1.0
AP4    1.0
AP5    1.0
AP6    1.0
AP7    1.0
AP8    1.0
AP9    1.0
dtype: float64

In [56]:
# define dataset class
class CustomDataset(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = torch.tensor(self.data[index], dtype=torch.float32)
        y = torch.tensor(self.target[index], dtype=torch.float32)
        return x, y

In [57]:
# create dataset objects
train_dataset = CustomDataset(X_train.values, y_train.values)
test_dataset = CustomDataset(X_test.values, y_test.values)

# create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [58]:
# define model class

# set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class MultiLabelBinaryClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

class MultiLabelBinaryClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.output = nn.Linear(64, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.layer1(x))
        x = self.sigmoid(self.output(x))
        return x

# define model and optimizer
model = MultiLabelBinaryClassifier(6, 9).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-1)

# define loss function
criterion = nn.BCELoss()

In [59]:
losses = []

# train the model
num_epochs = 1000
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    if epoch % 100 == 0:
        print(f"Epoch {epoch+1} loss: {running_loss / len(train_loader)}")
    losses.append(running_loss / len(train_loader))


Epoch 1 loss: 0.8276149272918701
Epoch 101 loss: 0.05957064678271611
Epoch 201 loss: 0.01969584518422683
Epoch 301 loss: 0.012013698865969976
Epoch 401 loss: 0.007393818038205306
Epoch 501 loss: 0.005546017196805527
Epoch 601 loss: 0.008820474070186416
Epoch 701 loss: 0.00468451227255476
Epoch 801 loss: 0.0029793719673762097
Epoch 901 loss: 0.0005610535328742117


In [60]:
# test the model
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        total += labels.shape[0] * labels.shape[1] # count number of labels
        correct += (predicted == labels).float().sum().item()

print(f"Accuracy: {100 * correct / total}%")

Accuracy: 98.71794871794872%
